import pickle
from sklearn.manifold import TSNE
import pandas as pd
import re
# ^^^ pyforest auto-imports - don't write above this line
# Imports

In [499]:
import folium
from collections import Counter
import postman_problems
import networkx as nx
import itertools
import dwave_networkx as dnx
import time
import dijkstar

## My Data

In [269]:
stations_df = pd.read_csv("./saved_data/final_station_df.csv", index_col = 0)

<IPython.core.display.Javascript object>

In [270]:
non_unique_stations_df = pd.read_csv("./saved_data/non_unique_mta_stations.csv", index_col=0)

<IPython.core.display.Javascript object>

In [271]:
my_edgelist = pd.read_csv('./saved_data/edge_list_df_no_req.csv', index_col=0)

<IPython.core.display.Javascript object>

In [272]:
node_list_df = pd.read_csv("./saved_data/nodelist_nyc_subway.csv", index_col=0).reset_index()

<IPython.core.display.Javascript object>

In [273]:
node_list_df

,station_id,X,Y
0,101,40.889248,-73.898583
1,103,40.884667,-73.900870
2,104,40.878856,-73.904834
3,106,40.874561,-73.909831
4,107,40.869444,-73.915279
...,...,...,...
439,R42,40.634967,-74.023377
440,R43,40.629742,-74.025510
441,R44,40.622687,-74.028398
442,S03,40.674772,-73.957624


### Checking my data

In [274]:
# every node is in node_list_df
node1 = my_edgelist.node1.tolist()
node2 = my_edgelist.node2.tolist()

In [275]:
all_nodes_edgelist = set(node1 + node2)

In [276]:
for x in node_list_df['station_id']:
    if x not in all_nodes_edgelist:
        print(x)

In [277]:
for x in node2:
    if 'D17' in x:
        print(x)

D17_R17
D17_R17


# Mini Dijkstar Attempt

## Getting A,C,E Edges

In [278]:
good_indices_ace = []
for idx, x in enumerate(my_edgelist['node1']):
    for letter in 'A':
        if letter in x and letter in my_edgelist['node2'][idx]:
            if idx not in good_indices_ace:
                good_indices_ace.append(idx)

In [279]:
A_edgelist = my_edgelist.iloc[good_indices_ace]

In [280]:
A_edgelist.reset_index(inplace=True, drop=True)

In [281]:
A_edgelist[A_edgelist['node1'] == '112_A09']

,node1,node2,trail,color,distance
5,112_A09,A10,nyc subway,red,90
6,112_A09,A12_D13,nyc subway,red,210


## Getting A,C,E Nodes

In [396]:
A_nodelist = node_list_df[node_list_df.station_id.str.contains("A")]
# C_nodelist = node_list_df[node_list_df.station_id.str.contains("C")]
# E_nodelist = node_list_df[node_list_df.station_id.str.contains("E")]
# A_nodelist = pd.concat([A_nodelist, C_nodelist])

In [397]:
A_nodelist.reset_index(inplace=True, drop=True)

In [398]:
A_nodelist.shape

(52, 3)

In [399]:
A_nodelist.head()

,station_id,X,Y
0,112_A09,40.840556,-73.940133
1,125_A24,40.768247,-73.981929
2,A02,40.868072,-73.919899
3,A03,40.865491,-73.927271
4,A05,40.859022,-73.934180


## Making TSP Matrix

In [400]:
dijk_ace_graph = dijkstar.Graph(undirected=True)

In [401]:
zipped_edges = list(zip(A_edgelist['node1'], A_edgelist['node2'], A_edgelist['distance']))
for x in zipped_edges:
    dijk_ace_graph.add_edge(x[0], x[1], x[2])

In [402]:
len(zipped_edges)

62

### Find distances between all pairs

In [403]:
def get_indirect_distance(dijkstar_graph, nodelist):
    distance_matrix = []
    for i in range(len(nodelist)):
        start_node = nodelist['station_id'][i]
        one_node_tree = []
        for num in range(i, len(nodelist)):
            dest_node = nodelist['station_id'][num]
            path = dijkstar.find_path(dijkstar_graph, start_node, dest_node)
            path_distance = path.total_cost
            one_node_tree.append(path_distance)
        distance_matrix.append(one_node_tree)  # top right triangle of matrix
    for i in range(len(distance_matrix)):
#         print(f"i = {i}")
        for x in range(i, len(distance_matrix)):
#             print(f"x = {x}")
            if distance_matrix[i][x] != 0:
                distance_matrix[x].insert(i, distance_matrix[i][x])
    return distance_matrix

In [404]:
A_dist_matrix = get_indirect_distance(dijk_ace_graph, A_nodelist)

In [405]:
len(A_dist_matrix)

52

In [406]:
len(A_dist_matrix[0])

52

In [407]:
len(A_dist_matrix[-1])

52

## Using OR Tools

**Note**: This code was adapted from the above python file developed by Google OR-Tools. License can be found here: https://www.apache.org/licenses/LICENSE-2.0

- This is trying to reach every stop and return to the original node

In [408]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(A_dist_matrix):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = A_dist_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(A_dist_matrix)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 9150 seconds
Route for vehicle 0:
 0 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 18 -> 1 -> 19 -> 20 -> 21 -> 22 -> 23 -> 24 -> 25 -> 26 -> 27 -> 28 -> 29 -> 30 -> 31 -> 32 -> 33 -> 34 -> 35 -> 36 -> 37 -> 38 -> 39 -> 40 -> 41 -> 42 -> 43 -> 44 -> 45 -> 46 -> 47 -> 48 -> 49 -> 50 -> 51 -> 6 -> 5 -> 4 -> 3 -> 2 -> 0



<IPython.core.display.Javascript object>

In [458]:
def get_time_in_hrs(seconds):
    minutes = seconds / 60
    hours = minutes / 60
    remainder_hour = hours % 1
    final_minutes = round((remainder_hour * 60), 2)
    return round(hours, 2), f"The entire route takes {hours - remainder_hour} hours, {final_minutes} minutes"

In [459]:
short_route_time_hours, short_route_time_message = get_time_in_hrs(seconds)
short_route_time_message

'The entire route takes 12.0 hours, 42.5 minutes'

### Translating list of stops to station_ids

In [445]:
desired_row = node_list_df[node_list_df['station_id'] == '716'].index
desired_row[0]
idx = [desired_row[0]] + [i for i in range(len(node_list_df)) if i != desired_row]
modified_nodelist = node_list_df.iloc[idx].reset_index(drop=True)
nodes = modified_nodelist['station_id']

In [447]:
def tsp_station_ids(list_of_stops, nodelist, starting_stop = '101'):
    if starting_stop == '101':
        nodes = nodelist['station_id']
        list_of_nodes = [nodes[int(x)] for x in list_of_stops]
    elif starting_stop != '101':
        desired_row = nodelist[nodelist['station_id'] == starting_stop].index
        idx = [desired_row[0]] + [i for i in range(len(nodelist)) if i != desired_row]
        modified_nodelist = nodelist.iloc[idx].reset_index(drop=True)
        nodes = modified_nodelist['station_id']
        list_of_nodes = [nodes[int(x)] for x in list_of_stops]
    return list_of_nodes

In [415]:
x = tsp_station_ids(list_of_stops, A_nodelist)

In [299]:
# x

### Getting station names in order

In [302]:
def get_station_names(stations_df, tsp_station_ids):
    full_list = []
    for mta_station in tsp_station_ids:
        single_stations = mta_station.split("_")
        station_string = ""
        counter = 0
        for station in single_stations:
            row = stations_df.index[stations_df['stop_id'] == station].tolist()
            for item in row:
                name = stations_df['stop_name'][item]
                if counter >= 1:
                    station_string += " / " + name
                    counter += 1
                elif counter == 0:
                    station_string += name
                    counter += 1
        full_list.append(station_string)
    return full_list

# Applying to All the Data

## Instantiating graph

In [303]:
nyc_graph = dijkstar.Graph(undirected=True)

In [304]:
nyc_zipped_edges = list(zip(my_edgelist['node1'], my_edgelist['node2'], my_edgelist['distance']))
for x in nyc_zipped_edges:
    nyc_graph.add_edge(x[0], x[1], x[2])

## Making Distance Matrix

In [309]:
nyc_dist_matrix = get_indirect_distance(nyc_graph, node_list_df)

In [310]:
len(nyc_dist_matrix)

444

### Make a key for each row so that I don't need to re-run this function

In [324]:
matrix_station_zip = list(zip(node_list_df['station_id'], nyc_dist_matrix))
dist_matrix_dictionary = {key:value for key, value in matrix_station_zip}

In [327]:
# list(dist_matrix_dictionary.items())[0]

### Function to put desired station first 

In [377]:
def make_station_first_in_matrix(matrix_dict, desired_stations_id):
    new_dict = {desired_stations_id: matrix_dict[desired_stations_id]}
#     new_dict.update({desired_stations_id: matrix_dict[desired_stations_id]})
    for x in matrix_dict:
        if x not in new_dict:
            new_dict.update({x: matrix_dict[x]})
    return new_dict

In [551]:
len(nyc_dist_matrix)

444

In [554]:
orig_dist_matrix = make_station_first_in_matrix(dist_matrix_dictionary, '101')
# orig_dist_matrix = make_dict_a_matrix(nyc_dist_matrix)

In [556]:
len(orig_dist_matrix)

444

### Turning Dict into matrix

In [385]:
def make_dict_a_matrix(dictionary):
    matrix_values = [value for value in dictionary.values()]
    return matrix_values

### Save Matrix Dict

In [563]:
# with open("./saved_data/Distance_matrix_as_dict.pickle", 'wb+') as f:
#     pickle.dump(orig_dist_matrix, f)

<IPython.core.display.Javascript object>

## Running Algorithm

In [311]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(nyc_dist_matrix):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = nyc_dist_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(nyc_dist_matrix)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 71070 seconds
Route for vehicle 0:
 0 -> 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 172 -> 173 -> 174 -> 175 -> 176 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 18 -> 19 -> 20 -> 17 -> 61 -> 60 -> 59 -> 58 -> 92 -> 91 -> 57 -> 56 -> 55 -> 54 -> 53 -> 52 -> 51 -> 50 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 113 -> 112 -> 111 -> 110 -> 109 -> 105 -> 114 -> 115 -> 116 -> 117 -> 118 -> 119 -> 120 -> 121 -> 122 -> 123 -> 124 -> 125 -> 126 -> 127 -> 128 -> 129 -> 130 -> 131 -> 132 -> 133 -> 134 -> 135 -> 136 -> 137 -> 138 -> 139 -> 140 -> 141 -> 143 -> 144 -> 145 -> 362 -> 193 -> 192 -> 191 -> 190 -> 189 -> 248 -> 283 -> 282 -> 167 -> 321 -> 322 -> 323 -> 324 -> 325 -> 326 -> 390 -> 391 -> 392 -> 327 -> 328 -> 329 -> 330 -> 331 -> 201 -> 202 -> 203 -> 340 -> 339 -> 338 -> 337 -> 336 -> 335 -> 344 -> 343 -> 342 -> 341 -> 334 -> 333 -> 332 -> 221 -> 220 -> 219 -> 218 -> 217 -> 216 -> 215 -> 214 -> 213 -> 212 -> 211 -> 210 -> 209 -> 208 -> 207 -> 20

<IPython.core.display.Javascript object>

In [457]:
short_route_time_hours, short_route_time_message = get_time_in_hrs(seconds)
short_route_time_hours

12.708333333333334

In [318]:
full_nyc_tsp_stations = tsp_station_ids(list_of_stops, node_list_df)

In [319]:
# full_nyc_tsp_stations

In [320]:
tsp_names = get_station_names(non_unique_stations_df, full_nyc_tsp_stations)

In [323]:
# tsp_names

# Finding Fastest TSP by Starting Station 

## Making DataFrame

In [508]:
TSP_solutions_df = pd.DataFrame(columns = ['Starting_station', "Station_id_route", "Station_name_route",
                                          'Time_seconds', "Time_hours"])

<IPython.core.display.Javascript object>

In [465]:
TSP_solutions_df.loc[0] = ["boy"]

## Formulas for adding to DataFrame

In [468]:
def results_to_df(time, stop_list, original_stop, stations_df=non_unique_stations_df, nodelistdf=node_list_df,
                  dist_matrix_dict=dist_matrix_dictionary, final_df=TSP_solutions_df):
    stop_dict = make_station_first_in_matrix(dist_matrix_dict, original_stop)
    stop_dist_matrix = make_dict_a_matrix(stop_dict)
    time_in_hours, time_message = get_time_in_hrs(time)
    tsp_stations = tsp_station_ids(stop_list, nodelistdf, starting_stop=original_stop)
    station_names = get_station_names(stations_df, tsp_stations)
    data = [original_stop, tsp_stations, station_names, time, time_in_hours]
    return data

## Choosing which stations to test

### Geographic center

In [416]:
absolute_difference_function3 = lambda list_value : abs(list_value[0] - lat_mean) + abs(list_value[1] - lon_mean)
closest_value3 = min(list(zip(node_list_df['X'], node_list_df['Y'])), key=absolute_difference_function3)

In [417]:
closest_value3

(40.744587, -73.93099699999998)

In [418]:
node_list_df[(node_list_df['X'] == closest_value3[0]) & (node_list_df['Y'] == closest_value3[1])]

,station_id,X,Y
165,716,40.744587,-73.930997


In [419]:
non_unique_stations_df[non_unique_stations_df['stop_id'] == '716']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
504,716,33 St,40.744587,-73.930997,1,NaN


#### Running Formulas

In [420]:
dict_716 = make_station_first_in_matrix(dist_matrix_dictionary, '716')

In [421]:
matrix_716 = make_dict_a_matrix(dict_716)

In [422]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(matrix_716):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix_716
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix_716)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 45750 seconds
Route for vehicle 0:
 0 -> 165 -> 164 -> 163 -> 162 -> 161 -> 160 -> 159 -> 158 -> 157 -> 156 -> 155 -> 154 -> 153 -> 152 -> 151 -> 150 -> 149 -> 148 -> 147 -> 146 -> 145 -> 144 -> 143 -> 142 -> 141 -> 140 -> 139 -> 138 -> 137 -> 136 -> 135 -> 134 -> 133 -> 132 -> 131 -> 130 -> 129 -> 128 -> 127 -> 126 -> 125 -> 124 -> 123 -> 122 -> 121 -> 120 -> 119 -> 118 -> 117 -> 116 -> 115 -> 114 -> 113 -> 112 -> 111 -> 110 -> 109 -> 108 -> 107 -> 106 -> 105 -> 104 -> 103 -> 102 -> 101 -> 100 -> 99 -> 98 -> 97 -> 96 -> 95 -> 94 -> 93 -> 92 -> 91 -> 90 -> 89 -> 88 -> 87 -> 86 -> 85 -> 84 -> 83 -> 82 -> 81 -> 80 -> 79 -> 78 -> 77 -> 76 -> 75 -> 74 -> 73 -> 72 -> 71 -> 70 -> 69 -> 68 -> 67 -> 66 -> 65 -> 64 -> 63 -> 62 -> 61 -> 60 -> 59 -> 58 -> 57 -> 56 -> 55 -> 54 -> 53 -> 52 -> 51 -> 50 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 37 -> 36 -> 35 -> 34 -> 33 -> 32 -> 31 -> 30 -> 29 -> 28 -> 27 -> 26 -> 25 -> 24 -> 23 -> 22 -> 21 -> 20 -> 19 -> 

<IPython.core.display.Javascript object>

In [461]:
short_route_time_hours, short_route_time_message  = get_time_in_hrs(seconds)
short_route_time_message

'The entire route takes 12.0 hours, 42.5 minutes'

In [448]:
tsp_stations_716 = tsp_station_ids(list_of_stops, node_list_df, starting_stop='716')

In [452]:
# tsp_stations_716

In [453]:
station_names_716 = get_station_names(non_unique_stations_df, tsp_stations_716)

#### Put into df

In [469]:
data_716 = results_to_df(seconds, list_of_stops, "716")

In [509]:
TSP_solutions_df.loc[0] = data_716

### Most Connections

#### Finding most_connected Stations

In [476]:
all_nodes_edglist_list = node1 + node2
node_count_dict = {i: all_nodes_edglist_list.count(i) for i in all_nodes_edgelist}

In [488]:
max(list(node_count_dict.values()))

10

#### Appears 10 times

In [496]:
most_connected_station = [k for k, v in Counter(node_count_dict.items()) if v > 9]

In [497]:
most_connected_station

['635_L03_R20']

In [501]:
non_unique_stations_df[non_unique_stations_df['stop_id'].str.contains("635")]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
447,635,14 St - Union Sq,40.734673,-73.989951,1,NaN


##### Solving it

In [502]:
dict_union_sq = make_station_first_in_matrix(dist_matrix_dictionary, '635_L03_R20')
matrix_union_sq = make_dict_a_matrix(dict_union_sq)

In [503]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(matrix_union_sqmatrix_716):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix_union_sq
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix_union_sq)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 48030 seconds
Route for vehicle 0:
 0 -> 146 -> 145 -> 144 -> 143 -> 142 -> 141 -> 140 -> 139 -> 138 -> 137 -> 136 -> 135 -> 134 -> 133 -> 132 -> 131 -> 130 -> 129 -> 128 -> 127 -> 126 -> 125 -> 124 -> 123 -> 122 -> 121 -> 120 -> 119 -> 118 -> 117 -> 116 -> 115 -> 114 -> 113 -> 112 -> 111 -> 110 -> 109 -> 108 -> 107 -> 106 -> 105 -> 104 -> 103 -> 102 -> 101 -> 100 -> 99 -> 98 -> 97 -> 96 -> 95 -> 94 -> 93 -> 92 -> 91 -> 90 -> 89 -> 88 -> 87 -> 86 -> 85 -> 84 -> 83 -> 82 -> 81 -> 80 -> 79 -> 78 -> 77 -> 76 -> 75 -> 74 -> 73 -> 72 -> 71 -> 70 -> 69 -> 68 -> 67 -> 66 -> 65 -> 64 -> 63 -> 62 -> 61 -> 60 -> 59 -> 58 -> 57 -> 56 -> 55 -> 54 -> 53 -> 52 -> 51 -> 50 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 37 -> 36 -> 35 -> 34 -> 33 -> 32 -> 31 -> 30 -> 29 -> 28 -> 27 -> 26 -> 25 -> 24 -> 23 -> 22 -> 21 -> 20 -> 19 -> 18 -> 17 -> 16 -> 15 -> 14 -> 13 -> 12 -> 11 -> 10 -> 176 -> 175 -> 174 -> 173 -> 172 -> 180 -> 181 -> 182 -> 183 -> 184 -> 185 -> 18

<IPython.core.display.Javascript object>

In [505]:
data_union_sq = results_to_df(seconds, list_of_stops, "635_L03_R20")

In [510]:
TSP_solutions_df.loc[1] = data_union_sq

#### Appears 9 times

In [513]:
most_connected_station_9 = [k for k, v in Counter(node_count_dict.items()) if v == 9]

In [514]:
most_connected_station_9

['127_725_902_R16', '235_D24_R31']

In [515]:
non_unique_stations_df[non_unique_stations_df['stop_id'].str.contains("127")]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
72,127,Times Sq - 42 St,40.75529,-73.987495,1,NaN


In [516]:
non_unique_stations_df[non_unique_stations_df['stop_id'].str.contains("235")]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
210,235,Atlantic Av - Barclays Ctr,40.684359,-73.977666,1,NaN


##### Times Sq

In [517]:
dict_times_sq = make_station_first_in_matrix(dist_matrix_dictionary, '127_725_902_R16')
matrix_times_sq = make_dict_a_matrix(dict_times_sq)

In [518]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(matrix_times_sq):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix_times_sq
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix_times_sq)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 70410 seconds
Route for vehicle 0:
 0 -> 24 -> 23 -> 22 -> 21 -> 20 -> 19 -> 18 -> 17 -> 16 -> 15 -> 14 -> 13 -> 12 -> 11 -> 10 -> 176 -> 175 -> 174 -> 173 -> 172 -> 190 -> 191 -> 192 -> 193 -> 362 -> 145 -> 144 -> 143 -> 142 -> 114 -> 115 -> 116 -> 117 -> 118 -> 119 -> 120 -> 121 -> 122 -> 123 -> 124 -> 125 -> 126 -> 127 -> 128 -> 129 -> 130 -> 131 -> 91 -> 92 -> 61 -> 60 -> 59 -> 58 -> 109 -> 110 -> 111 -> 112 -> 113 -> 38 -> 39 -> 40 -> 41 -> 42 -> 43 -> 44 -> 45 -> 46 -> 47 -> 48 -> 49 -> 50 -> 51 -> 52 -> 53 -> 54 -> 55 -> 56 -> 57 -> 105 -> 132 -> 133 -> 134 -> 135 -> 136 -> 137 -> 138 -> 139 -> 140 -> 141 -> 169 -> 168 -> 320 -> 319 -> 318 -> 317 -> 316 -> 315 -> 160 -> 161 -> 162 -> 163 -> 164 -> 165 -> 166 -> 416 -> 415 -> 414 -> 413 -> 412 -> 411 -> 417 -> 418 -> 419 -> 420 -> 171 -> 194 -> 195 -> 274 -> 196 -> 197 -> 198 -> 397 -> 396 -> 395 -> 394 -> 393 -> 254 -> 284 -> 285 -> 286 -> 287 -> 199 -> 288 -> 289 -> 290 -> 291 -> 292 -> 293 -> 294 -> 295 -> 296 -> 29

<IPython.core.display.Javascript object>

In [519]:
data_times_sq = results_to_df(seconds, list_of_stops, "127_725_902_R16")

In [520]:
TSP_solutions_df.loc[2] = data_times_sq

##### Barclays Ctr

In [522]:
dict_barclays = make_station_first_in_matrix(dist_matrix_dictionary, '235_D24_R31')
matrix_barclays = make_dict_a_matrix(dict_barclays)

In [523]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(matrix_barclays):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix_barclays
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix_barclays)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 63660 seconds
Route for vehicle 0:
 0 -> 69 -> 68 -> 67 -> 66 -> 65 -> 64 -> 63 -> 62 -> 61 -> 60 -> 59 -> 92 -> 91 -> 58 -> 57 -> 56 -> 55 -> 54 -> 53 -> 52 -> 51 -> 50 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 37 -> 36 -> 35 -> 34 -> 33 -> 32 -> 31 -> 30 -> 29 -> 28 -> 27 -> 26 -> 25 -> 24 -> 23 -> 22 -> 21 -> 20 -> 19 -> 18 -> 17 -> 16 -> 15 -> 14 -> 13 -> 12 -> 11 -> 10 -> 176 -> 175 -> 174 -> 173 -> 172 -> 180 -> 181 -> 182 -> 183 -> 184 -> 185 -> 186 -> 187 -> 188 -> 170 -> 142 -> 143 -> 144 -> 145 -> 108 -> 107 -> 106 -> 151 -> 150 -> 149 -> 148 -> 147 -> 422 -> 421 -> 251 -> 250 -> 249 -> 225 -> 410 -> 409 -> 408 -> 224 -> 223 -> 222 -> 160 -> 315 -> 316 -> 317 -> 318 -> 319 -> 320 -> 417 -> 418 -> 419 -> 420 -> 171 -> 252 -> 253 -> 194 -> 195 -> 274 -> 196 -> 197 -> 198 -> 397 -> 396 -> 395 -> 394 -> 393 -> 254 -> 284 -> 285 -> 286 -> 255 -> 362 -> 193 -> 192 -> 191 -> 190 -> 189 -> 248 -> 283 -> 282 -> 167 -> 321 -> 322 -> 323 -> 32

<IPython.core.display.Javascript object>

In [524]:
data_barclays = results_to_df(seconds, list_of_stops, "235_D24_R31")

In [525]:
TSP_solutions_df.loc[3] = data_barclays

In [526]:
TSP_solutions_df

,Starting_station,Station_id_route,Station_name_route,Time_seconds,Time_hours
0,716,"[716, 715, 714, 713, 712, 711, 710_G14, 709, 7...","[33 St, 40 St, 46 St, 52 St, Woodside - 61 St,...",45750,12.71
1,635_L03_R20,"[635_L03_R20, 634, 633, 632, 631_723_901, 630,...",[14 St - Union Sq / Union Sq - 14 St / 14 St -...,48030,13.34
2,127_725_902_R16,"[127_725_902_R16, 126, 125_A24, 124, 123, 122,...",[Times Sq - 42 St / Times Sq - 42 St / Times S...,70410,19.56
3,235_D24_R31,"[235_D24_R31, 234, 233, 232_423, 231, 230, 229...",[Atlantic Av - Barclays Ctr / Atlantic Av - Ba...,63660,17.68


### Edge node (control)

#### Far Rockaway

In [528]:
# starting from what I hypothesize to be the worst place to start and end the subway challenge from
non_unique_stations_df[non_unique_stations_df['stop_name'].str.contains("Far Rockaway")]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
1071,H11,Far Rockaway - Mott Av,40.603995,-73.755405,1,NaN


In [529]:
dict_rockaway = make_station_first_in_matrix(dist_matrix_dictionary, 'H11')
matrix_rockaway = make_dict_a_matrix(dict_rockaway)

In [530]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(matrix_rockaway):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix_rockaway
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix_rockaway)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 19500 seconds
Route for vehicle 0:
 0 -> 340 -> 339 -> 338 -> 337 -> 336 -> 335 -> 341 -> 342 -> 343 -> 344 -> 334 -> 333 -> 332 -> 331 -> 330 -> 329 -> 328 -> 327 -> 326 -> 325 -> 324 -> 323 -> 322 -> 321 -> 320 -> 319 -> 318 -> 317 -> 316 -> 315 -> 314 -> 313 -> 312 -> 311 -> 310 -> 309 -> 308 -> 307 -> 306 -> 305 -> 304 -> 303 -> 302 -> 301 -> 300 -> 299 -> 298 -> 297 -> 296 -> 295 -> 294 -> 293 -> 292 -> 291 -> 290 -> 289 -> 288 -> 287 -> 286 -> 285 -> 284 -> 283 -> 282 -> 281 -> 280 -> 279 -> 278 -> 277 -> 276 -> 275 -> 274 -> 273 -> 272 -> 271 -> 270 -> 269 -> 268 -> 267 -> 266 -> 265 -> 264 -> 263 -> 262 -> 261 -> 260 -> 259 -> 258 -> 257 -> 256 -> 255 -> 254 -> 253 -> 252 -> 251 -> 250 -> 249 -> 248 -> 247 -> 246 -> 245 -> 244 -> 243 -> 242 -> 241 -> 240 -> 239 -> 238 -> 237 -> 236 -> 235 -> 234 -> 233 -> 232 -> 231 -> 230 -> 229 -> 228 -> 227 -> 226 -> 225 -> 408 -> 409 -> 410 -> 224 -> 223 -> 222 -> 221 -> 220 -> 219 -> 218 -> 217 -> 216 -> 215 -> 214 -> 213 -> 212

<IPython.core.display.Javascript object>

In [531]:
data_rockaway = results_to_df(seconds, list_of_stops, "H11")

In [532]:
TSP_solutions_df.loc[4] = data_rockaway

In [533]:
TSP_solutions_df

,Starting_station,Station_id_route,Station_name_route,Time_seconds,Time_hours
0,716,"[716, 715, 714, 713, 712, 711, 710_G14, 709, 7...","[33 St, 40 St, 46 St, 52 St, Woodside - 61 St,...",45750,12.71
1,635_L03_R20,"[635_L03_R20, 634, 633, 632, 631_723_901, 630,...",[14 St - Union Sq / Union Sq - 14 St / 14 St -...,48030,13.34
2,127_725_902_R16,"[127_725_902_R16, 126, 125_A24, 124, 123, 122,...",[Times Sq - 42 St / Times Sq - 42 St / Times S...,70410,19.56
3,235_D24_R31,"[235_D24_R31, 234, 233, 232_423, 231, 230, 229...",[Atlantic Av - Barclays Ctr / Atlantic Av - Ba...,63660,17.68
4,H11,"[H11, H10, H09, H08, H07, H06, H04_H19, H12, H...","[Far Rockaway - Mott Av, Beach 25 St, Beach 36...",19500,5.42


In [540]:
TSP_solutions_df['Station_name_route'][4][-5:]

['Marble Hill - 225 St',
 '231 St',
 '238 St',
 'Van Cortlandt Park - 242 St',
 'Far Rockaway - Mott Av']

#### Pelham Bay

In [542]:
# starting from what I hypothesize to be the worst place to start and end the subway challenge from
non_unique_stations_df[non_unique_stations_df['stop_name'].str.contains("Pelham Bay")]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
351,601,Pelham Bay Park,40.852462,-73.828121,1,NaN


In [543]:
dict_pelham = make_station_first_in_matrix(dist_matrix_dictionary, '601')
matrix_pelham = make_dict_a_matrix(dict_pelham)

In [544]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(matrix_pelham):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix_pelham
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(matrix_pelham)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 52290 seconds
Route for vehicle 0:
 0 -> 114 -> 113 -> 112 -> 111 -> 110 -> 109 -> 108 -> 107 -> 106 -> 105 -> 104 -> 103 -> 102 -> 101 -> 100 -> 99 -> 98 -> 97 -> 96 -> 95 -> 94 -> 93 -> 92 -> 91 -> 90 -> 89 -> 88 -> 87 -> 86 -> 85 -> 84 -> 83 -> 82 -> 81 -> 80 -> 79 -> 78 -> 77 -> 76 -> 75 -> 74 -> 73 -> 72 -> 71 -> 70 -> 69 -> 68 -> 67 -> 66 -> 65 -> 64 -> 63 -> 62 -> 61 -> 60 -> 59 -> 58 -> 57 -> 56 -> 55 -> 54 -> 53 -> 52 -> 51 -> 50 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 37 -> 36 -> 35 -> 34 -> 33 -> 32 -> 31 -> 30 -> 29 -> 28 -> 27 -> 26 -> 25 -> 24 -> 23 -> 22 -> 21 -> 20 -> 19 -> 18 -> 17 -> 16 -> 15 -> 14 -> 13 -> 12 -> 11 -> 10 -> 176 -> 175 -> 174 -> 173 -> 172 -> 238 -> 239 -> 240 -> 241 -> 242 -> 243 -> 244 -> 245 -> 246 -> 115 -> 116 -> 117 -> 118 -> 119 -> 120 -> 121 -> 122 -> 123 -> 124 -> 125 -> 126 -> 127 -> 128 -> 129 -> 130 -> 131 -> 132 -> 133 -> 134 -> 135 -> 136 -> 137 -> 138 -> 139 -> 140 -> 141 -> 142 -> 143 -> 14

<IPython.core.display.Javascript object>

In [545]:
data_pelham = results_to_df(seconds, list_of_stops, "601")

In [546]:
TSP_solutions_df.loc[5] = data_pelham

In [547]:
TSP_solutions_df

,Starting_station,Station_id_route,Station_name_route,Time_seconds,Time_hours
0,716,"[716, 715, 714, 713, 712, 711, 710_G14, 709, 7...","[33 St, 40 St, 46 St, 52 St, Woodside - 61 St,...",45750,12.71
1,635_L03_R20,"[635_L03_R20, 634, 633, 632, 631_723_901, 630,...",[14 St - Union Sq / Union Sq - 14 St / 14 St -...,48030,13.34
2,127_725_902_R16,"[127_725_902_R16, 126, 125_A24, 124, 123, 122,...",[Times Sq - 42 St / Times Sq - 42 St / Times S...,70410,19.56
3,235_D24_R31,"[235_D24_R31, 234, 233, 232_423, 231, 230, 229...",[Atlantic Av - Barclays Ctr / Atlantic Av - Ba...,63660,17.68
4,H11,"[H11, H10, H09, H08, H07, H06, H04_H19, H12, H...","[Far Rockaway - Mott Av, Beach 25 St, Beach 36...",19500,5.42
5,601,"[601, 505, 504, 503, 502, 501, 420, 419, 418, ...","[Pelham Bay Park, Morris Park, Pelham Pkwy, Gu...",52290,14.53


## Saving df 

In [548]:
# TSP_solutions_df.to_csv("./saved_data/TSP_solutions_df.csv")